In [1]:
import math

class RMK_Analysis_Engine:
    """
    This class is the complete and definitive implementation of the RMK Project,
    a multi-layered system for analyzing the performance of regimes and the
    well-being of societies throughout history.
    """

    # --- Constants as defined in the RMK Codex ---
    GOVERNANCE_BASE = 200000 / 3
    CORRUPTION_CONSTANT = 80
    HV_SCALING_CONSTANT = 2.68 # Retained for historical context in the notes.

    def __init__(self):
        """Initializes the engine with global crime data for equivalence calculations."""
        self.global_murders = 458000
        self.crime_data = {
            "rape": 1400000,
            "kidnapping": 300000,
            "robbery": 1300000,
            "grievous_hurt": 15000000,
            "human_trafficking": 50000
        }

    # --- Core Mathematical Functions ---

    def _get_governance_mark(self, perish_rate):
        """Calculates the Governance Mark (x) from a given Perish Rate."""
        if perish_rate <= 0:
            return 100.0
        try:
            value = 1 / perish_rate
            return (math.log(value) / math.log(self.GOVERNANCE_BASE)) * 100
        except (ValueError, ZeroDivisionError):
            return 0 # Catastrophic failure results in zero.

    def _get_governance_ratio(self, perish_rate):
        """Calculates the Governance Ratio from a Perish Rate."""
        mark = self._get_governance_mark(perish_rate)
        return mark / 100

    def _get_crime_equivalent_multiplier(self, crime_name):
        """Gets the perfected 'Equivalent Death' multiplier for a non-lethal crime."""
        if crime_name not in self.crime_data or self.crime_data[crime_name] == 0:
            return 0

        # The final, perfected formula from the RMK Codex.
        multiplier = self.global_murders / self.crime_data[crime_name]

        # The crucial capping rule.
        return min(multiplier, 1.0)

    # --- Pillar 1: APPORASDT (The Ruler's Performance) ---

    def calculate_regime_governance(self, regime_data):
        """
        Calculates the final, absolute Governance Ratio for a single regime.
        This is a multi-stage process as per the perfected RMK codex.
        """
        # Step 1: Calculate Initial GR (without corruption)
        initial_deaths = regime_data.get('initial_deaths_due_to_regime', 0)
        ccm = regime_data.get('ccm', 1) # Avoid division by zero

        initial_perish_rate = initial_deaths / ccm if ccm > 0 else 0
        initial_gr = self._get_governance_ratio(initial_perish_rate)

        # Step 2: Calculate Corruption Deaths using the Initial GR
        corruption_ratio = regime_data.get('corruption_ratio', 0)
        zenith_population = regime_data.get('zenith_population', 0)

        # This is the perfected, sequential formula
        corruption_deaths = (corruption_ratio * (1 - initial_gr) / self.CORRUPTION_CONSTANT * zenith_population)

        # Step 3: Calculate Final GR
        total_deaths_due_to_regime = initial_deaths + corruption_deaths

        final_perish_rate = total_deaths_due_to_regime / ccm if ccm > 0 else 0

        # Handle the special case where deaths > CCM
        if total_deaths_due_to_regime >= ccm:
            final_perish_rate = total_deaths_due_to_regime / (ccm + total_deaths_due_to_regime)

        return self._get_governance_ratio(final_perish_rate)

    def calculate_apporasdt(self, current_regime_gr, previous_regime_gr):
        """Calculates the final, relative APPORASDT score."""
        if previous_regime_gr <= 0:
            # If the previous regime was a total failure, any stability is a massive improvement.
            return 0.99

        non_hostile_ratio = (current_regime_gr ** 2) / previous_regime_gr

        # The rule for when a new regime is a massive improvement.
        if non_hostile_ratio > 1:
            return non_hostile_ratio / (non_hostile_ratio + previous_regime_gr)
        else:
            return non_hostile_ratio

    # --- Pillar 2: HV (The Citizen's Reality) ---

    def calculate_hv(self, societal_data):
        """Calculates the final Happiness Variable (HV) for a society."""
        # Step 1: Calculate Direct and Equivalent Deaths
        direct_deaths = societal_data.get('direct_deaths', 0)
        equivalent_deaths = 0
        for crime, count in societal_data.get('crime_counts', {}).items():
            multiplier = self._get_crime_equivalent_multiplier(crime)
            equivalent_deaths += count * multiplier

        total_death_toll = direct_deaths + equivalent_deaths
        annual_mortality = societal_data.get('annual_mortality', 1)

        # Step 2: Calculate Happiness Index Score
        societal_perish_rate = total_death_toll / annual_mortality if annual_mortality > 0 else 0
        if total_death_toll >= annual_mortality:
             societal_perish_rate = total_death_toll / (annual_mortality + total_death_toll)

        happiness_index_score = self._get_governance_ratio(societal_perish_rate)

        # Step 3: Calculate Final HV using the perfected formula
        avg_tax_percentage = societal_data.get('avg_tax_percentage', 1.0) # Assume 100% if not given
        if avg_tax_percentage <= 0: avg_tax_percentage = 0.01 # Avoid division by zero

        initial_hv = happiness_index_score / avg_tax_percentage

        # Step 4: Apply the HV > 1 Update Rule
        if initial_hv > 1:
            return initial_hv / (initial_hv + happiness_index_score)
        else:
            return initial_hv

    # --- Pillar 3: The Grand Unified Theory ---

    def calculate_well_being_index(self, apporasdt, hv):
        """Calculates the final Well-Being Index."""
        return (apporasdt + hv) / 2

    def get_wbi_tier(self, wbi_score):
        """Returns the qualitative interpretation of a WBI score."""
        if wbi_score >= 0.80: return "Tier V: The Thriving Society"
        if wbi_score >= 0.60: return "Tier IV: The Functional Society"
        if wbi_score >= 0.40: return "Tier III: The Struggling Society"
        if wbi_score >= 0.20: return "Tier II: The Failing Society"
        return "Tier I: The Collapsed Society"

# --- Example Usage: Analyzing Finland (c. 2023) ---

# Initialize the engine
engine = RMK_Analysis_Engine()

# 1. Calculate Finland's HV (Citizen's Reality)
finland_society_data = {
    "annual_mortality": 57200,
    "direct_deaths": 1255, # Murders, Suicides, Accidents, etc.
    "crime_counts": {
        "rape": 1500,
        "robbery": 1800,
        "kidnapping": 50
    },
    "avg_tax_percentage": 0.43
}
finland_hv = engine.calculate_hv(finland_society_data)

# 2. Calculate Finland's APPORASDT (Ruler's Performance)
# We need to analyze both current and previous regimes.
# For Finland, they are functionally identical, stable, low-corruption democracies.
finland_regime_data = {
    "initial_deaths_due_to_regime": 0,
    "ccm": 57200, # Using 1 year for simplicity
    "corruption_ratio": 0.01,
    "zenith_population": 5500000
}
finland_current_gr = engine.calculate_regime_governance(finland_regime_data)
finland_previous_gr = finland_current_gr # As they are identical in performance
finland_apporasdt = engine.calculate_apporasdt(finland_current_gr, finland_previous_gr)


# 3. Calculate the Final Well-Being Index
finland_wbi = engine.calculate_well_being_index(finland_apporasdt, finland_hv)
finland_tier = engine.get_wbi_tier(finland_wbi)


# --- Print the Definitive Report ---
print("--- RMK Project: Definitive Analysis of Finland (c. 2023) ---")
print(f"\nThe Citizen's Reality (HV): {finland_hv:.2%}")
print(f"The Ruler's Performance (APPORASDT): {finland_apporasdt:.2%}")
print("-" * 60)
print(f"THE WELL-BEING INDEX (WBI): {finland_wbi:.2%}")
print(f"CLASSIFICATION: {finland_tier}")
print("\n--- Analysis Complete ---")

--- RMK Project: Definitive Analysis of Finland (c. 2023) ---

The Citizen's Reality (HV): 66.13%
The Ruler's Performance (APPORASDT): 100.00%
------------------------------------------------------------
THE WELL-BEING INDEX (WBI): 83.07%
CLASSIFICATION: Tier V: The Thriving Society

--- Analysis Complete ---


In [2]:
import math

class RMK_Analysis_Engine:
    """
    This class is the complete and definitive implementation of the RMK Project,
    a multi-layered system for analyzing the performance of regimes and the
    well-being of societies throughout history. This is the interactive version.
    """

    # --- Constants as defined in the RMK Codex ---
    GOVERNANCE_BASE = 200000 / 3
    CORRUPTION_CONSTANT = 80

    def __init__(self):
        """Initializes the engine with global crime data for equivalence calculations."""
        self.global_murders = 458000
        self.crime_data = {
            "rape": 1400000,
            "kidnapping": 300000,
            "robbery": 1300000,
            "grievous_hurt": 15000000,
            "human_trafficking": 50000
        }

    # --- Core Mathematical Functions (Unchanged) ---

    def _get_governance_mark(self, perish_rate):
        if perish_rate <= 0: return 100.0
        try:
            value = 1 / perish_rate
            return (math.log(value) / math.log(self.GOVERNANCE_BASE)) * 100
        except (ValueError, ZeroDivisionError): return 0

    def _get_governance_ratio(self, perish_rate):
        mark = self._get_governance_mark(perish_rate)
        return mark / 100

    def _get_crime_equivalent_multiplier(self, crime_name):
        if crime_name not in self.crime_data or self.crime_data[crime_name] == 0: return 0
        multiplier = self.global_murders / self.crime_data[crime_name]
        return min(multiplier, 1.0)

    # --- Interactive Data Input ---

    def get_input(self, prompt, input_type=float):
        """A robust function to get and validate user input."""
        while True:
            try:
                return input_type(input(prompt))
            except ValueError:
                print("Invalid input. Please enter a valid number.")

    # --- Pillar 1: APPORASDT (The Ruler's Performance) ---

    def get_regime_data(self, regime_name):
        """Interactively collects all data needed for a single regime analysis."""
        print(f"\n--- Entering Data for {regime_name} ---")
        duration = self.get_input("Enter the duration of the regime in years: ")
        avg_annual_mortality = self.get_input("Enter the average annual mortality during the regime: ")

        initial_deaths = self.get_input("Enter the TOTAL initial deaths (wars, purges, riots) over the entire reign: ")
        corruption_ratio = self.get_input("Enter the Corruption Ratio (e.g., 0.10 for 10%): ")
        zenith_population = self.get_input("Enter the zenith (maximum) population under the regime: ")

        return {
            "name": regime_name,
            "ccm": avg_annual_mortality * duration,
            "initial_deaths_due_to_regime": initial_deaths,
            "corruption_ratio": corruption_ratio,
            "zenith_population": zenith_population
        }

    def calculate_regime_governance(self, regime_data):
        initial_perish_rate = regime_data['initial_deaths_due_to_regime'] / regime_data['ccm'] if regime_data['ccm'] > 0 else 0
        initial_gr = self._get_governance_ratio(initial_perish_rate)

        corruption_deaths = (regime_data['corruption_ratio'] * (1 - initial_gr) / self.CORRUPTION_CONSTANT * regime_data['zenith_population'])

        total_deaths = regime_data['initial_deaths_due_to_regime'] + corruption_deaths

        final_perish_rate = total_deaths / regime_data['ccm'] if regime_data['ccm'] > 0 else 0
        if total_deaths >= regime_data['ccm']:
            final_perish_rate = total_deaths / (regime_data['ccm'] + total_deaths)

        return self._get_governance_ratio(final_perish_rate)

    def calculate_apporasdt(self, current_regime_gr, previous_regime_gr):
        if previous_regime_gr <= 0: return 0.99
        non_hostile_ratio = (current_regime_gr ** 2) / previous_regime_gr
        if non_hostile_ratio > 1:
            return non_hostile_ratio / (non_hostile_ratio + previous_regime_gr)
        else:
            return non_hostile_ratio

    # --- Pillar 2: HV (The Citizen's Reality) ---

    def get_societal_data(self):
        """Interactively collects all data needed for an HV calculation."""
        print("\n--- Entering Data for Societal Happiness (HV) ---")
        annual_mortality = self.get_input("Enter the Annual Mortality for the society: ")

        print("\nEnter Annual Direct Deaths:")
        murders = self.get_input("  Murders: ")
        suicides = self.get_input("  Suicides: ")
        accidents = self.get_input("  Accident Deaths: ")
        narcotics = self.get_input("  Narcotic Deaths: ")
        poverty = self.get_input("  Poverty-Related Deaths: ")
        anti_regime = self.get_input("  Deaths from Anti-Regime/Terror Forces: ")

        direct_deaths = murders + suicides + accidents + narcotics + poverty + anti_regime

        print("\nEnter Annual NON-LETHAL Crime Counts:")
        crime_counts = {}
        for crime in self.crime_data.keys():
            count = self.get_input(f"  {crime.capitalize()}: ")
            crime_counts[crime] = count

        avg_tax_percentage = self.get_input("\nEnter the Average Tax Percentage (as a decimal, e.g., 0.27 for 27%): ")

        return {
            "annual_mortality": annual_mortality,
            "direct_deaths": direct_deaths,
            "crime_counts": crime_counts,
            "avg_tax_percentage": avg_tax_percentage
        }

    def calculate_hv(self, societal_data):
        equivalent_deaths = sum(count * self._get_crime_equivalent_multiplier(crime) for crime, count in societal_data['crime_counts'].items())
        total_death_toll = societal_data['direct_deaths'] + equivalent_deaths

        societal_perish_rate = total_death_toll / societal_data['annual_mortality'] if societal_data['annual_mortality'] > 0 else 0
        if total_death_toll >= societal_data['annual_mortality']:
             societal_perish_rate = total_death_toll / (societal_data['annual_mortality'] + total_death_toll)

        happiness_index_score = self._get_governance_ratio(societal_perish_rate)

        avg_tax_percentage = societal_data['avg_tax_percentage']
        if avg_tax_percentage <= 0: avg_tax_percentage = 0.01

        initial_hv = happiness_index_score / avg_tax_percentage

        if initial_hv > 1:
            return initial_hv / (initial_hv + happiness_index_score)
        else:
            return initial_hv

    # --- Pillar 3: The Grand Unified Theory ---

    def calculate_well_being_index(self, apporasdt, hv):
        return (apporasdt + hv) / 2

    def get_wbi_tier(self, wbi_score):
        if wbi_score >= 0.80: return "Tier V: The Thriving Society"
        if wbi_score >= 0.60: return "Tier IV: The Functional Society"
        if wbi_score >= 0.40: return "Tier III: The Struggling Society"
        if wbi_score >= 0.20: return "Tier II: The Failing Society"
        return "Tier I: The Collapsed Society"

    def get_apporasdt_tier(self, apporasdt_score):
        if apporasdt_score >= 0.90: return "Benevolent / Savior"
        if apporasdt_score >= 0.70: return "Popular / Effective"
        if apporasdt_score >= 0.50: return "Contentious / Status Quo"
        if apporasdt_score >= 0.30: return "Unpopular / Failing"
        if apporasdt_score >= 0.10: return "Detested / Oppressive"
        return "Tyrannical / Genocidal"

    # --- The Main Program Loop ---

    def run_analysis(self):
        """The main interactive loop to run a full RMK analysis."""
        print("="*60)
        print("      WELCOME TO THE DEFINITIVE RMK ANALYSIS ENGINE")
        print("="*60)

        # Part 1: APPORASDT Calculation
        print("\n--- STAGE 1: CALCULATING APPORASDT (The Ruler's Performance) ---")
        previous_regime_data = self.get_regime_data("the PREVIOUS Regime")
        current_regime_data = self.get_regime_data("the CURRENT Regime")

        previous_gr = self.calculate_regime_governance(previous_regime_data)
        current_gr = self.calculate_regime_governance(current_regime_data)

        apporasdt = self.calculate_apporasdt(current_gr, previous_gr)

        # Part 2: HV Calculation
        print("\n--- STAGE 2: CALCULATING HV (The Citizen's Reality) ---")
        societal_data = self.get_societal_data()
        hv = self.calculate_hv(societal_data)

        # Part 3: Final Report
        wbi = self.calculate_well_being_index(apporasdt, hv)

        print("\n\n" + "="*60)
        print("                 FINAL RMK ANALYSIS REPORT")
        print("="*60)
        print("\n--- THE RULER'S PERFORMANCE ---")
        print(f"  Previous Regime Absolute Governance Ratio: {previous_gr:.2%}")
        print(f"  Current Regime Absolute Governance Ratio: {current_gr:.2%}")
        print(f"  FINAL APPORASDT SCORE: {apporasdt:.2%}")
        print(f"  Classification: '{self.get_apporasdt_tier(apporasdt)}'")

        print("\n--- THE CITIZEN'S REALITY ---")
        print(f"  FINAL HAPPINESS VARIABLE (HV): {hv:.2%}")
        print(f"  Classification: '{self.get_apporasdt_tier(hv)}'") # HV is comparable to APPORASDT

        print("\n" + "-"*60)
        print(f"  THE WELL-BEING INDEX (WBI): {wbi:.2%}")
        print(f"  FINAL ARCHETYPE: {self.get_wbi_tier(wbi)}")
        print("="*60)


# --- Run the Engine ---
if __name__ == "__main__":
    engine = RMK_Analysis_Engine()
    engine.run_analysis()


      WELCOME TO THE DEFINITIVE RMK ANALYSIS ENGINE

--- STAGE 1: CALCULATING APPORASDT (The Ruler's Performance) ---

--- Entering Data for the PREVIOUS Regime ---
Enter the duration of the regime in years: 2
Enter the average annual mortality during the regime: 1000000
Enter the TOTAL initial deaths (wars, purges, riots) over the entire reign: 10000
Enter the Corruption Ratio (e.g., 0.10 for 10%): 0.25
Enter the zenith (maximum) population under the regime: 50000000

--- Entering Data for the CURRENT Regime ---
Enter the duration of the regime in years: 2
Enter the average annual mortality during the regime: 1000000
Enter the TOTAL initial deaths (wars, purges, riots) over the entire reign: 100
Enter the Corruption Ratio (e.g., 0.10 for 10%): 0.1
Enter the zenith (maximum) population under the regime: 50000000

--- STAGE 2: CALCULATING HV (The Citizen's Reality) ---

--- Entering Data for Societal Happiness (HV) ---
Enter the Annual Mortality for the society: 1000000

Enter Annual Di